In [ ]:
pip install openai langchain langgraph pandas jira langchain-openai


In [ ]:
%pip install markdown
%pip install --upgrade pandas

In [ ]:
import os
from pathlib import Path
from dotenv import load_dotenv, find_dotenv, dotenv_values

# Define the path to the environment file
env_file_path = Path("oneConnection.env")

# Load environment variables from the specific file
load_dotenv(env_file_path, override=True)  # This loads variables into os.environ

# Print only the keys in a nicely formatted way
env_vars = dotenv_values(env_file_path)  # Loads environment variables without adding them to os.environ
print("Environment Variables in oneConnection.env:")
environment_variables = list(env_vars.keys()) # OR env_vars.values
# print(environment_variables)

In [ ]:
# Connect to JIRA
from jira import JIRA

# Get Jira credentials
jira_email = os.getenv('JIRA_EMAIL')
jira_api_token = os.getenv('JIRA_API_TOKEN')
jira_url = os.getenv('JIRA_URL')
jira_project = os.getenv('JIRA_PROJECT')

jira_options = {'server': jira_url}
jira = JIRA(options=jira_options, basic_auth=(jira_email, jira_api_token))

issues = jira.search_issues(f'project={jira_project}', maxResults=1000)

# Print the number of issues found
print(f"Number of issues found: {len(issues)}\n")

In [ ]:
# Get JIRA issues data and put it in a DataFrame

import pandas as pd

jira_data = []
for issue in issues:
    # Extract comments from the issue
    comments = []
    # Ensure comments are loaded
    issue_comments = jira.comments(issue)
    for comment in issue_comments:
        comments.append(comment.body)
    
    jira_data.append({
        'Key': issue.key,
        'Summary': issue.fields.summary,
        'Description': issue.fields.description if issue.fields.description else '',
        'Comments': comments,
        # 'Status': issue.fields.status.name,
        # 'Assignee': issue.fields.assignee.displayName if issue.fields.assignee else 'Unassigned',
        # 'Reporter': issue.fields.reporter.displayName,
        # 'Created': issue.fields.created[:10],  # Extract date
    })

jira_df = pd.DataFrame(jira_data)
display(jira_df)

In [ ]:
# Connect to Miro
# Extract all text-related items from a Miro board, including all pages. 
# The resulting DataFrame should contain all text from shapes, sticky notes, and text items present on the board.

import requests
import json

# Load your access token from an environment variable or secure storage
access_token = os.getenv('MIRO_ACCESS_TOKEN')

# Ensure the access token is available
if not access_token:
    raise ValueError("MIRO_ACCESS_TOKEN environment variable is not set")

headers = {
    'Authorization': f'Bearer {access_token}',
    'Content-Type': 'application/json'
}

# Specify the board name to extract data from
target_board_name = os.getenv('MIRO_BOARD')  # Replace with your board name

# Ensure the board name is provided
if not target_board_name:
    raise ValueError("MIRO_BOARD environment variable is not set")

# Get list of boards
url = 'https://api.miro.com/v2/boards'
response = requests.get(url, headers=headers)

miro_data = []

if response.status_code == 200:
    boards = response.json()['data']
    
    # Find the target board by name
    target_board = next((board for board in boards if board['name'] == target_board_name), None)

    if target_board:
        board_id = target_board['id']
        board_name = target_board['name']

        # Initialize pagination
        items_url = f'https://api.miro.com/v2/boards/{board_id}/items'
        
        while items_url:
            items_response = requests.get(items_url, headers=headers)

            if items_response.status_code == 200:
                items_data = items_response.json()
                
                # Extract items data
                items = items_data.get('data', [])
                
                for item in items:
                    # Extract both sticky notes, text boxes, and any shapes that have content
                    if item['type'] in ['sticky_note', 'text', 'shape']:
                        content = item['data'].get('content', "")
                        if content:  # Only add items that have content
                            miro_data.append({
                                'Board ID': board_id,
                                'Board Name': board_name,
                                'Item Type': item['type'],
                                'Content': content
                            })
                
                # Check for the next page link
                next_link = items_data.get('links', {}).get('next', None)
                items_url = next_link if next_link else None

            else:
                print(f"Failed to retrieve items for board {board_id}: {items_response.status_code}")
                break
    else:
        print(f"No board found with the name '{target_board_name}'")

else:
    print(f"Failed to retrieve boards: {response.status_code}")
    print(response.text)

# Create DataFrame from the extracted data
miro_df = pd.DataFrame(miro_data)

# Display the DataFrame using standard Pandas
display(miro_df)


In [ ]:
strategy_template = """
Our company's strategic objective is:

*Integrating Security Posture Management into Observability*
Description:
With the increasing frequency of cyber threats, integrating security into all stages of the software lifecycle is crucial. Dynatrace can expand its platform to include security posture management, embedding security monitoring into the observability stack to support DevSecOps practices.
Strategic Objectives:
Unified DevSecOps Platform: Merge performance monitoring with real-time security analytics to provide a holistic view of application health and security.
Automated Threat Detection: Utilize AI to detect and respond to security threats automatically, minimizing response times.
Compliance and Governance: Help organizations meet regulatory requirements by providing detailed security reports and compliance checks.
Education and Training: Offer resources and training to help teams adopt DevSecOps methodologies effectively.
Benefits:
Reduced Security Risks: Early detection of vulnerabilities and threats helps prevent breaches and data loss.
Faster Time-to-Market: Integrating security into development accelerates deployment cycles by reducing rework and delays caused by security issues.
Cost Savings: Preventing security incidents can save significant costs associated with breaches, fines, and reputational damage.
"""

product_description = """
Dynatrace is a leading software intelligence company that offers an all-in-one observability platform designed to simplify cloud complexity and accelerate digital transformation. 
The Dynatrace platform leverages artificial intelligence (AI) and automation to provide deep insights into applications, infrastructure, and user experiences, enabling organizations to optimize performance, innovate faster, and deliver exceptional digital services.

Key Features of Dynatrace:
1. Full-Stack Observability:
Unified Monitoring: Provides end-to-end visibility across the entire technology stack, including applications, microservices, containers, Kubernetes, infrastructure, and networks.
Contextual Data: Captures metrics, logs, traces, and real user data in a single platform for comprehensive analysis.
2. AI-Powered Insights:
Davis AI Engine: Utilizes advanced AI to automatically detect anomalies, identify root causes, and predict potential issues before they impact users.
Problem Resolution: Reduces alert noise by focusing on root causes rather than symptoms, enabling faster remediation.
3. Automated Monitoring:
Auto-Discovery: Automatically detects and instruments applications and services without manual configuration.
Continuous Updates: Keeps monitoring configurations up-to-date in dynamic cloud environments.
4. Cloud-Native Support:
Multi-Cloud Compatibility: Integrates seamlessly with AWS, Azure, Google Cloud Platform, and hybrid cloud infrastructures.
Container and Orchestration Support: Monitors Docker, Kubernetes, and serverless environments effectively.
5. Application Performance Management (APM):
Code-Level Visibility: Provides deep insights into application performance, including transaction tracing and code-level diagnostics.
Real-Time Monitoring: Tracks application health in real-time to ensure optimal performance.
6. Digital Experience Monitoring (DEM):
User Experience Insights: Monitors real user interactions to assess performance from the end-user perspective.
Synthetic Monitoring: Simulates user transactions to proactively identify performance issues.
7. Infrastructure Monitoring:
Resource Optimization: Monitors hosts, processes, and network health to optimize resource utilization.
Anomaly Detection: Identifies infrastructure anomalies that could affect application performance.
8. DevOps Integration:
Shift-Left Monitoring: Integrates with CI/CD pipelines to detect issues early in the development cycle.
Collaboration Tools: Supports integrations with platforms like Jira, Slack, and ServiceNow for streamlined workflows.
9. Security Posture Management:
Runtime Application Protection: Identifies vulnerabilities and protects applications at runtime without additional agents.
Unified Observability and Security: Combines monitoring and security insights within a single platform.
10. Open Platform and Extensibility:
APIs and Extensions: Offers a range of APIs and community-supported extensions to customize and extend platform capabilities.
Third-Party Integrations: Connects with various tools and technologies to enhance existing workflows.

"""

prompt_template = """
You are a product management assistant.

Given the Strategic topic and the Product description, analyze the Customer feedback.

**Instructions:**

1. **Relevance Check**: Determine if the Customer feedback is directly and tightly related to the Strategic topic.\n
If it is not relevant, respond with "Not relevant."

2. **Opportunity Identification**: If the Customer feedback is relevant:

   - **a. Summarize** the relevant points from the Customer feedback.
   - **b. Identify** opportunities for improvement or new features that align with the Strategic topic.


**Output Format:**

- **If not relevant:**

- **If relevant:**
    -- *Relevance check:*
    -- *Summary:*
    -- *Opportunities:*


**Strategic topic:**

{strategy_template}

**Product description:**

{product_description}

**Customer feedback:**

{customer_feedback}
"""


from langchain.prompts import ChatPromptTemplate
import openai
from langchain_openai import ChatOpenAI
from langchain.schema.runnable import RunnableSequence

# Initialize the LLM
openai_api_key = os.getenv('OPENAI_API_KEY')

llm = ChatOpenAI(
    openai_api_key=openai_api_key,
    temperature=0.2,
    model="gpt-4o",
    top_p=1.0,  
    presence_penalty=0 
)

template = ChatPromptTemplate.from_template(prompt_template)
# chain = LLMChain(llm=llm, prompt=template)

# Use RunnableSequence to chain the prompt and the LLM
chain = template | llm


In [ ]:
from IPython.display import display, HTML
import markdown

# Jira opportunities extraction
jira_opportunities = []
for index, row in jira_df.iterrows():
    # Prepare customer feedback
    comments_text = ' '.join(row['Comments']) if row['Comments'] else ''
    feedback = f"{row['Summary']}\n\n{row['Description']}\n\n{comments_text}"
    
    # Input data for the LLM
    input_data = {
        'strategy_template': strategy_template,
        'product_description': product_description,
        'customer_feedback': feedback
    }
    
    try:
        # Run the chain with invoke
        result_message = chain.invoke(input_data)  # Result is an AIMessage object
        result = result_message.content.strip()  # Extract text content
        
        # Check if the feedback is relevant
        if 'Not relevant' in result:
            continue  # Skip adding this issue
        else:
            # Add to opportunities list
            jira_opportunities.append({
                'Key': row['Key'],
                'Summary': row['Summary'],
                'Opportunities': result
            })
    except Exception as e:
        print(f"Error processing issue {row['Key']}: {e}")
        jira_opportunities.append({
            'Key': row['Key'],
            'Summary': row['Summary'],
            'Opportunities': f"Error: {e}"
        })

# Create the DataFrame
jira_opportunities_df = pd.DataFrame(jira_opportunities)

# Adjust pandas display options
pd.set_option('display.max_colwidth', None)

# Function to clean and convert markdown to HTML
def clean_and_convert_markdown(text):
    if isinstance(text, str):
        # Replace escape sequences
        text = text.replace('\\n', '\n').replace('\\t', '\t')
        # Convert markdown to HTML
        html = markdown.markdown(text)
        return html
    else:
        return text

# Apply the function to the 'Opportunities' column
jira_opportunities_df['Opportunities'] = jira_opportunities_df['Opportunities'].apply(clean_and_convert_markdown)

# Use pandas Styler to render HTML content
styled_df = (
    jira_opportunities_df.style
    .format({'Opportunities': lambda x: x})
    .set_properties(subset=['Opportunities'], **{'text-align': 'left'})
    .hide(axis='index')  # Use 'hide' with 'axis' parameter
)

# Render the DataFrame with HTML content
jira_html = styled_df.to_html(escape=False)

# Display the DataFrame
display(HTML("<h2>Jira Opportunities</h2>"))
display(HTML(jira_html))


In [ ]:
# Miro opportunities extraction
miro_opportunities = []
for index, row in miro_df.iterrows():
    # Prepare feedback data
    feedback = row['Content']

    # Input data for the LLM
    input_data = {
        'strategy_template': strategy_template,
        'product_description': product_description,
        'customer_feedback': feedback
    }

    try:
        # Run the chain
        result = chain.invoke(input_data)

        # Extract text from AIMessage object
        result_text = result.content.strip()

        # Check if the feedback is relevant
        if 'Not relevant' in result_text:
            continue  # Skip adding this content
        else:
            # Add to opportunities list, excluding the "Board ID" column
            miro_opportunities.append({
                'Board Name': row['Board Name'],
                'Item Type': row['Item Type'],
                'Content': row['Content'],
                'Opportunities': result_text
            })
    except Exception as e:
        print(f"Error processing content from board {row['Board Name']}: {e}")
        miro_opportunities.append({
            'Board Name': row['Board Name'],
            'Item Type': row['Item Type'],
            'Content': row['Content'],
            'Opportunities': f"Error: {e}"
        })

# Create the DataFrame from the opportunities list
miro_opportunities_df = pd.DataFrame(miro_opportunities)

# Adjust pandas display options
pd.set_option('display.max_colwidth', None)

# Function to clean and convert markdown to HTML
def clean_and_convert_markdown(text):
    if isinstance(text, str):
        # Replace escape sequences
        text = text.replace('\\n', '\n').replace('\\t', '\t')
        # Convert markdown to HTML
        html = markdown.markdown(text)
        return html
    else:
        return text

# Apply the function to the 'Opportunities' column
miro_opportunities_df['Opportunities'] = miro_opportunities_df['Opportunities'].apply(clean_and_convert_markdown)

# Use pandas Styler to render HTML content
styled_df = (
    miro_opportunities_df.style
    .format({'Opportunities': lambda x: x})
    .set_properties(subset=['Opportunities'], **{'text-align': 'left'})
    .hide(axis='index')
)

# Render the DataFrame with HTML content
miro_html = styled_df.to_html(escape=False)

# Display the DataFrame
display(HTML("<h2>Miro Opportunities</h2>"))
display(HTML(miro_html))


In [ ]:
# Define prompt template for comparing opportunities. 
comparison_prompt = """
You are an assistant that helps identify if two opportunities have a matching context.
Below are two opportunities:

Opportunity 1: "{opportunity_1}"
Opportunity 2: "{opportunity_2}"

Are these opportunities similar in context? If they are, reply with "Yes" and provide a short explanation of why. If not, reply with "No".
"""

template = ChatPromptTemplate.from_template(comparison_prompt)

# Use RunnableSequence to chain the prompt and the LLM
chain = template | llm

# Prepare opportunities from Miro and Jira
miro_opportunities = miro_opportunities_df[['Opportunities', 'Board Name', 'Item Type']].to_dict('records')
jira_opportunities = jira_opportunities_df[['Opportunities', 'Key']].to_dict('records')

# List to hold final combined opportunities
combined_opportunities = []

# Compare Miro opportunities with Jira opportunities using LLM
for miro_opportunity in miro_opportunities:
    linked = False
    for jira_opportunity in jira_opportunities:
        input_data = {
            'opportunity_1': miro_opportunity['Opportunities'],
            'opportunity_2': jira_opportunity['Opportunities']
        }
        
        try:
            # Run the comparison using chain.invoke() to handle AIMessage properly
            result = chain.invoke(input_data)
            response = result.content.strip().lower()
            
            if response.startswith("yes"):
                linked = True
                
                # Add line breaks to the explanation for better formatting
                formatted_explanation = response.replace('yes ', '').capitalize()
                formatted_explanation = formatted_explanation.replace('. ', '.\n')
                
                combined_opportunities.append({
                    'Description': miro_opportunity['Opportunities'],
                    'Tool': f"Jira ({jira_opportunity['Key']}), Miro ({miro_opportunity['Item Type']})",
                    'Explanation': formatted_explanation  # Improved formatting
                })
                break  # Once matched, no need to compare further
                
        except Exception as e:
            print(f"Error comparing opportunities: {e}")

    # If no overlapping opportunity found, add the Miro row to the combined list
    if not linked:
        combined_opportunities.append({
            'Description': miro_opportunity['Opportunities'],
            'Tool': f"Miro ({miro_opportunity['Item Type']})"
        })

# Add Jira opportunities that didn't find a match in Miro to the combined list
for jira_opportunity in jira_opportunities:
    found_in_combined = any(jira_opportunity['Opportunities'] in opportunity['Description'] for opportunity in combined_opportunities)
    
    if not found_in_combined:
        combined_opportunities.append({
            'Description': jira_opportunity['Opportunities'],
            'Tool': f"Jira ({jira_opportunity['Key']})"
        })

# Create a DataFrame from the combined opportunities list
final_opportunities_df = pd.DataFrame(combined_opportunities)

# Adjust pandas display options
pd.set_option('display.max_colwidth', None)

# Function to clean and convert markdown to HTML with improved formatting for explanations
def clean_and_convert_markdown(text):
    if isinstance(text, str):
        # Replace escape sequences
        text = text.replace('\\n', '\n').replace('\\t', '\t')
        
        # Ensure each sentence in Explanation starts on a new line
        if text.lower().startswith('both') or text.lower().startswith('opportunity'):
            text = text.replace('. ', '.\n')
        
        # Convert markdown to HTML
        html = markdown.markdown(text)
        return html
    else:
        return text

# Apply the function to the 'Description' and 'Explanation' columns
final_opportunities_df['Description'] = final_opportunities_df['Description'].apply(clean_and_convert_markdown)
final_opportunities_df['Explanation'] = final_opportunities_df['Explanation'].apply(lambda x: clean_and_convert_markdown(x) if isinstance(x, str) else x)

# Use pandas Styler to render HTML content for the final DataFrame
styled_final_df = (
    final_opportunities_df.style
    .format({'Description': lambda x: x, 'Explanation': lambda x: x})
    .set_properties(subset=['Description', 'Explanation'], **{'text-align': 'left'})
    .hide(axis='index')
)

# Render the DataFrame with HTML content
html_final = styled_final_df.to_html(escape=False)

# Display the DataFrame
display(HTML("<h2>Combined Opportunities from Miro and Jira</h2>"))
display(HTML(html_final))
